In [43]:
import torch
import numpy as np
import time

In [132]:
in_image = torch.randn(1,5,5)
nn = torch.nn.Conv2d(1, 3, 3, padding=0, stride=1)

In [133]:
out_image = nn(in_image)

In [134]:
out_channels = nn.out_channels
in_channels = nn.in_channels
in_height = in_image.size(1)
in_width = in_image.size(2)
kernel_size = nn.kernel_size[0]
out_height = out_image.size(1)
out_width = out_image.size(2)
weights = nn.weight.detach()
stride = nn.stride[0]
padding = nn.padding[0]
bias = nn.bias
print(weights.size())

torch.Size([3, 1, 3, 3])


In [139]:
out_image.size()

torch.Size([3, 3, 3])

In [131]:
now=time.time()

padded_size = list(in_image.size())
padded_size = [ub_mult.size(0)] + padded_size
padded_size[2] = padded_size[2] + 2 * padding
padded_size[3] = padded_size[3] + 2 * padding

ub_mult = torch.reshape(ub_mult, (ub_mult.size(0),out_channels, out_height, out_width))
im = torch.zeros(padded_size)
print(im.size())
print(ub_mult.size())

for i_out in range(out_channels):
    channel_weights = weights[i_out]
    print(channel_weights.size())
    for i_h in range(out_height):
        for i_w in range(out_width):
            el = ub_mult[:,i_out,i_h,i_w]
            print(el.size())
            im[:,:,stride*i_h:stride*i_h+kernel_size, stride*i_w:stride*i_w+kernel_size] += el.view(-1,1,1,1) * channel_weights
            
if padding > 0:
    im = im[:,padding:-padding,padding:-padding]
print(time.time()-now)


torch.Size([100000, 3, 66, 66])
torch.Size([100000, 6, 32, 32])
torch.Size([3, 3, 3])
torch.Size([100000])
torch.Size([100000])
torch.Size([100000])
torch.Size([100000])


KeyboardInterrupt: 

In [93]:
ub_mult.shape

sum=ub_mult.sum(dim=[2,3])
bias_r = bias.view(1,-1)
(sum * bias_r).sum(1)



torch.Size([10, 6])
torch.Size([1, 6])
tensor([48.1798, 48.1798, 48.1798, 48.1798, 48.1798, 48.1798, 48.1798, 48.1798,
        48.1798, 48.1798], grad_fn=<SumBackward1>)


tensor([ 0.0013,  0.1298, -0.1485,  ..., -0.2031,  0.1664, -0.0834],
       grad_fn=<UnsafeViewBackward0>)

In [147]:

now=time.time()
in_width_p = in_width + padding * 2
in_height_p = in_height + padding * 2

size_p = in_height_p * in_width_p
in_dim = size_p * in_channels
out_dim = out_height * out_width * out_channels
res = torch.zeros((out_dim, in_dim))

# build row fillers
len_rows = (in_channels - 1) * size_p + (kernel_size - 1) * in_width_p + kernel_size
channels = torch.zeros((out_channels, len_rows))

for i_out in range(out_channels):
    for i_in in range(in_channels):
        i_p = i_in * size_p
        for k in range(kernel_size):
            start = i_p + k * in_width_p
            end = start + kernel_size
            channels[i_out, start:end] = weights[i_out, i_in, k]
            print(channels)

    for i_out_height in range(out_height):
        for i_out_width in range(out_width):
            start = i_out_height * stride * in_width_p + i_out_width * stride
            end = start + len_rows
            output = i_out * out_height * out_width + i_out_height * out_width + i_out_width
            res[output, start:end] = channels[i_out]

# remove padding
padding_rows = []
for i_in in range(in_channels):
    for i_in_height in range(in_height_p):
        for i_in_width in range(in_width_p):
            if i_in_width < padding or i_in_width >= padding + in_width:
                padding_rows.append(i_in * size_p + i_in_height * in_width_p + i_in_width)

        if i_in_height < padding or i_in_height >= padding + in_height:
            start = i_in * size_p + i_in_height * in_width_p
            end = start + in_width_p
            padding_rows = padding_rows + list(range(start, end))

padding_rows = list(np.unique(np.array(padding_rows)))  # delete duplicates

lc = torch.from_numpy(np.delete(res.numpy(), padding_rows, axis=1)).detach()

if bias is None:
    ret_bias = torch.zeros(out_width * out_height * out_channels)
else:
    ret_bias = torch.repeat_interleave(bias, out_width * out_height)
print(time.time()-now)

tensor([[-0.0385, -0.0475, -0.0795,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])
tensor([[-0.0385, -0.0475, -0.0795,  0.0000,  0.0000, -0.2651, -0.1284, -0.2169,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])
tensor([[-0.0385, -0.0475, -0.0795,  0.0000,  0.0000, -0.2651, -0.1284, -0.2169,
          0.0000,  0.0000, -0.3018,  0.0530, -0.0255],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  

In [148]:

lc.transpose(0,1)


tensor([[-0.0385,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0814,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000, -0.2883,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [-0.0475, -0.0385,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0953,  0.0814,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000, -0.2479, -0.2883,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [-0.0795, -0.0475, -0.0385,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000, -0.2766,  0.0953,  0.0814,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000, -0.2666, -0.2479, -0.2883,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.0795, -0.0475,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000, -0.2766,  0.0953,  0.0000,  0.0000,  0.0000,  0.0000

In [146]:
nn.weight

Parameter containing:
tensor([[[[-0.0385, -0.0475, -0.0795],
          [-0.2651, -0.1284, -0.2169],
          [-0.3018,  0.0530, -0.0255]]],


        [[[ 0.0814,  0.0953, -0.2766],
          [ 0.1166, -0.3143,  0.1683],
          [-0.2526,  0.1169, -0.0217]]],


        [[[-0.2883, -0.2479, -0.2666],
          [-0.3239,  0.1171,  0.2155],
          [ 0.2793,  0.0767,  0.2851]]]], requires_grad=True)

In [102]:
class DP_Conv2D(nn.Module):
    def _init_(self, inner: nn.Conv2d, input_shape):
        super()._init_()

        self.in_channels = inner.in_channels
        self.out_channels = inner.out_channels

        self.kernel_size = inner.kernel_size[0]
        self.stride = inner.stride[0]
        self.padding = inner.padding[0]
        
        self.in_width = int(sqrt(input_shape / self.in_channels)) # did not work for colour channels 
        self.out_height= int((self.in_width + 2 * self.padding - self.kernel_size) / self.stride + 1)
        self.out_features = (self.out_height**2) * self.out_channels

        self.weight = inner.weight.detach().clone()
        self.matrix = self.build_matrix()

        if inner.bias != None:
            self.bias = inner.bias.detach().clone()
            self.bias_matrix = torch.repeat_interleave(self.bias, int(self.out_features/self.out_channels))
            
        else:
            self.bias_matrix = torch.zeros(self.out_features)

        # print(self.matrix.shape)
        # assert self.matrix.shape[1] == self.bias_matrix.shape[0], 'fuckup with dims in conv matrix layers'

    def build_matrix(self):
        concat_weights = np.zeros((1,self.in_width**2 * self.in_channels))  

        for i in range(self.out_channels):
            # loop over output channels and then append to the lower side

            weights_per_in_channel = []
            for j in range(self.in_channels):
                # loop over input channels and then append to the right side
                # print(i,j)
                    
                kernel_mat = self.weight[i,j,:,:]
                idx = np.arange((self.in_width+ 2*self.padding)**2)

                # cut away first rows of padding
                cond1 = idx < (self.in_width + 2*self.padding) * self.padding

                # cut away lower row of padding
                cond2 = idx >= (self.in_width + 2*self.padding) * (self.in_width + self.padding)

                # cut away left columns of padding
                cond3 = idx % (self.in_width + 2*self.padding) < self.padding

                # cut away right columns of padding
                cond4 = idx % (self.in_width + 2*self.padding) >= (self.in_width+ self.padding)

                remove_padding = np.logical_or.reduce(
                    (cond1, cond2, cond3, cond4)
                )

                # right columns not considered by kernel
                cond5 = idx % (self.in_width + 2*self.padding) > ((self.in_width+ 2*self.padding) - self.kernel_size)

                # lower rows not considered by kernel
                cond6 = idx >= (self.in_width + 2*self.padding) * (self.in_width + 2*self.padding - self.kernel_size + 1)

                if self.stride > 0:
                    cond7 = idx % (self.in_width + 2*self.padding) % self.stride != 0
                    cond8 = idx // (self.in_width + 2*self.padding)  % self.stride != 0
                else:
                    cond7 = np.zeros_like(cond1).astype(bool)
                    cond8 = np.zeros_like(cond1).astype(bool)

                remove_rows = np.logical_or.reduce(
                    (cond5, cond6, cond7, cond8)
                )
                
                row_toep = np.vstack(
                    (
                        np.hstack(
                            (
                                kernel_mat, 
                                np.zeros(
                                    (
                                        self.kernel_size, 
                                        self.in_width + 2*self.padding - self.kernel_size
                                    )
                                )
                            )
                        ),
                        np.zeros(
                            (
                                self.in_width + 2*self.padding - self.kernel_size, 
                                self.in_width + 2*self.padding
                            )
                        )
                    )
                ).reshape(-1)

                toep_mat = row_toeplitz(row_toep)
                weights = toep_mat[~remove_rows, :][:, ~remove_padding]
                weights_per_in_channel.append(weights)

            if self.in_channels > 1:
                # first concatenate all the weights -> direction before we append it to np array
                concat_weights = np.vstack((
                    concat_weights, 
                    np.concatenate(tuple(weights_per_in_channel), axis=1)
                ))
            else:
                # we have to write the loop otherwise concatenate will fail
                concat_weights = np.vstack((concat_weights, weights))

        return torch.tensor(concat_weights[1:,:], dtype = torch.float32)
      
    def forward(self, x: DeepPoly):

        lb_weights, ub_weights = self.matrix, self.matrix
        lb_bias, ub_bias = self.bias_matrix, self.bias_matrix

        new_lb_weights, new_lb_bias, new_ub_weights, new_ub_bias = backsub(
            lb_weights,
            lb_bias,
            ub_weights,
            ub_bias,
            parent = x
        )

        return DeepPoly(new_lb_weights, new_lb_bias, new_ub_weights, new_ub_bias, parent=x)
#        return DeepPoly(lb_weights, lb_bias, ub_weights, ub_bias, parent=x)

RuntimeError: inconsistent tensor size, expected tensor [614400] and src [6144] to have the same number of elements, but got 614400 and 6144 elements respectively